In [65]:
!pip install pypdf


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [73]:

# imports necessários

#langchains
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda, RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_text_splitters import TextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

#outras coisas
from typing import Optional
from pydantic import BaseModel, Field
#importando arquivos de configuração e apis

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


# Desafios de Langchain

Pedi uma lista de seis exercícios pro Claude e ele me retornou.

Vamos a eles

## Desafio 1
Crie uma chain que recebe o nome de um país e retorna 3 curiosidades sobre ele

In [2]:
country_template = ChatPromptTemplate.from_template("Cite 3 curiosidades sobre {country}")
country_model = ChatOpenAI(model='gpt-4o-mini')
country_parser = StrOutputParser()

country_chain = country_template | country_model | country_parser
country_chain.invoke({"country":"Austrália"})

'Claro! Aqui estão três curiosidades interessantes sobre a Austrália:\n\n1. **Biodiversidade Única**: A Austrália é conhecida por sua biodiversidade única e possui mais de 80% de suas plantas, mamíferos, répteis e anfíbios que não são encontrados em nenhum outro lugar do mundo. Animais icônicos como cangurus, coalas e o diabo-da-tasmânia são nativos do país.\n\n2. **O Uluru**: Também conhecido como Ayers Rock, o Uluru é uma gigantesca formação rochosa localizada no Território do Norte. Ele é um local sagrado para os povos indígenas Anangu e tem importância cultural significativa. O Uluru muda de cor durante o dia, especialmente ao amanhecer e ao pôr do sol, criando paisagens deslumbrantes.\n\n3. **Cultura e Esportes**: A Austrália é famosa por sua paixão por esportes, com o críquete, o rúgbi e o futebol australiano sendo muito populares. Além disso, o país possui uma rica herança cultural que inclui tradições indígenas que datam de mais de 65.000 anos, sendo os Aborígenes e os Torres S

## Desafio 2

Crie duas chains: a primeira gera uma receita a partir de uma lista de ingredientes, a segunda avalia a receita e sugere melhorias

In [4]:
recipe_template = ChatPromptTemplate.from_template("Me sugira uma receita com os seguintes ingredientes: {ingredientes}")
model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()

recipe_chain = recipe_template | model | parser

ingredientes = [
    "Arroz",
    "Batata",
    "Carne Moída"
]

#receita = recipe_chain.invoke({"ingredientes":ingredientes})

improve_recipe_template = ChatPromptTemplate.from_template("Sugira melhorias para a seguinte receita: {receita}")

improve_recipe_chain = improve_recipe_template | model | parser
#improve_recipe_chain.invoke({"receita":receita})

generate_recipe_chain = recipe_chain | improve_recipe_chain
generate_recipe_chain.invoke({"ingredientes":ingredientes})


'A receita de "Arroz de Carne Moída com Batata" já é deliciosa, mas algumas melhorias podem torná-la ainda mais saborosa e interessante. Aqui estão algumas sugestões:\n\n### Sugestões de Melhorias:\n\n1. **Adicionar Vegetais**: Inclua legumes como cenoura picada, ervilhas ou pimentão para incrementar a receita e adicionar nutrientes. Esses vegetais podem ser refogados junto com a cebola.\n\n2. **Temperos Adicionais**: Experimente adicionar temperos como açafrão, páprica ou até mesmo um pouco de molho shoyu ou molho de tomate na etapa do refogado da carne. Isso adiciona profundidade de sabor.\n\n3. **Caldo em vez de Água**: Utilize caldo de carne ou caldo de legumes em vez de água para cozinhar o arroz. O caldo vai enriquecer o sabor do prato, tornando-o mais saboroso.\n\n4. **Ervas Frescas e Especiarias**: Acrescente ervas frescas como tomilho ou alecrim, que combinam bem com carne moída e batata. Você pode pular as folhas de louro ou até usá-las juntamente.\n\n5. **Queijo**: Antes de 

## Desafio 3 

Crie uma chain que recebe a descrição de um problema, classifica a categoria (bug, feature request, dúvida) usando structured output com Pydantic, e depois gera uma resposta adequada ao tipo

In [34]:
class IdentifyRequest(BaseModel):
    """"Recebe uma string de usuário e retorna a categoria"""""
    category: str = Field(description="Define a categoria da mensagem do usuário")

model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()

received_request_template = ChatPromptTemplate.from_template("Você receberá uma mensagem de um usuário e deve classificá-la ou como bug ou feature request ou dúvida: {mensagem}")
received_request_template

chat_identify_request = model.with_structured_output(IdentifyRequest)
chat_identify_request

mensagem = "O botão de logar deveria ser azul" 

identify_request_chain = received_request_template | chat_identify_request 

process_request_template = ChatPromptTemplate.from_template("Você recebe a seguinte mensagem do usuário: '{mensagem}'. Essa mensagem foi categorizada como {category}. Forneça uma resposta adequada ao usuário")

process_request_chain = process_request_template | model | parser

runnable_identify_request = RunnableParallel(category=identify_request_chain | (lambda x: x.category), mensagem=lambda x: x['mensagem'])

request_chain = runnable_identify_request | process_request_chain

request_chain.invoke({"mensagem":mensagem})


'Olá!\n\nAgradecemos pelo seu feedback! Sua sugestão de que o botão de logar seja azul é muito importante para nós. Vamos avaliar essa possibilidade e considerar como podemos aprimorar a experiência do usuário. Fique à vontade para compartilhar mais ideias ou sugestões.\n\nObrigado por ajudar a melhorar nosso serviço!'

## Desafio 4


Expanda o exercício anterior: dependendo da categoria classificada, a chain deve seguir por caminhos diferentes. Bug vai pra um prompt que pede passos de reprodução, feature request vai pra um prompt que pede justificativa de negócio, dúvida vai pra um prompt que responde diretamente. 

In [35]:
# definir os prompts e chains de acordo com a categorização

default_chain = RunnableLambda(lambda _: "Não entendi.")

bug_prompt_template = ChatPromptTemplate.from_template("Leia o bug report e gere uma resposta ao usuário pedindo para explicar como reproduzir o problema: {mensagem}")
bug_chain = bug_prompt_template | model | parser

feature_request_template = ChatPromptTemplate.from_template("Leia a feature request do usuário e gere uma resposta pedindo uma justificativa ao usuário para a solicitação: {mensagem}")
feature_request_chain = feature_request_template | model | parser

question_template = ChatPromptTemplate.from_template("Leia a dúvida e procure responder a dúvida do usuário: {mensagem}. Caso não saiba, diga que não sabe mesmo")
question_chain = question_template | model | parser

branch_chain = RunnableBranch(
    (lambda x: x.get("category") == "bug", bug_chain),
    (lambda x: x.get("category") == "feature request", feature_request_chain),
    (lambda x: x.get("category") == "duvida", question_chain),
    default_chain
)

new_chain = runnable_identify_request | branch_chain 
new_chain.invoke({"mensagem":mensagem})

'Olá! Agradecemos por compartilhar sua sugestão sobre a cor do botão de logar. Poderia nos fornecer mais detalhes sobre o motivo pelo qual você acredita que ele deveria ser azul? Sua justificativa nos ajudará a entender melhor sua perspectiva e a considerar a alteração de forma mais eficaz. Obrigado!'

## Desafio 5

Crie um chatbot que mantém contexto de conversa. O usuário pode pedir recomendações de filmes, e o bot deve lembrar o que já recomendou e quais preferências o usuário mencionou

In [62]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um cinéfilo que ajuda usuários indicando filmes para os usuários, baseado no que eles pediram"),
    MessagesPlaceholder(variable_name="historico"),
    ("human","{duvida}")
])

model = ChatOpenAI(model='gpt-4o-mini')

main_chain = chat_prompt_template | model

memory = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = InMemoryChatMessageHistory()
    return memory[session_id]

chain_with_history = RunnableWithMessageHistory(
    runnable=main_chain,
    get_session_history=get_session_history,
    input_messages_key="duvida",
    history_messages_key="historico"
)

complete_chain = chain_with_history | parser
complete_chain.invoke({
    "duvida":"Me cite 5 filmes de terror"
},{"configurable":{"session_id":"user123"}})



'Claro! Aqui estão 5 filmes de terror que você pode gostar:\n\n1. **O Exorcista (1973)** - Um clássico do terror que conta a história de uma jovem possuída por um demônio e os esforços de um padre para exorcizá-la.\n\n2. **A Bruxa (2015)** - Ambientado na Nova Inglaterra do século XVII, o filme retrata uma família que enfrenta forças malignas em uma floresta próxima.\n\n3. **Hereditário (2018)** - Um filme perturbador que explora temas de luto e trauma familiar, à medida que uma família descobre segredos sombrios após a morte da matriarca.\n\n4. **It: A Coisa (2017)** - Baseado no livro de Stephen King, este filme conta a história de um grupo de crianças que enfrenta uma entidade maligna que se disfarça como um palhaço.\n\n5. **O Babadook (2014)** - Uma mãe e seu filho se deparam com um livro misterioso que parece dar vida a uma entidade aterrorizante, refletindo o luto e a paternidade.\n\nEspero que você encontre algo que goste!'

In [63]:
complete_chain.invoke({
    "duvida":"Qual destes é o mais assustador?"
},{"configurable":{"session_id":"user123"}})

'Entre os filmes que mencionei, **Hereditário (2018)** é frequentemente considerado um dos mais assustadores. O filme é elogiado por sua atmosfera pesada, construção de tensão e temas profundos que lidam com perda e trauma familiar. Muitas pessoas acham que ele causa um medo psicológico duradouro, que vai além de apenas sustos momentâneos.\n\nNo entanto, o impacto do terror pode variar de pessoa para pessoa, então se você aprecia um terror mais sobrenatural, também pode achar **O Exorcista** ou **A Bruxa** extremamente assustadores. Se você gosta de terror psicológico, **O Babadook** pode ser bastante perturbador. A escolha depende do que mais lhe atrai no gênero!'

## Desafio 6

Crie uma chain que carrega um documento PDF, divide em chunks, armazena num vector store (pode usar FAISS ou Chroma), e responde perguntas sobre o conteúdo do documento. Esse exercício junta embeddings, retriever e chain de QA, e é o padrão mais demandado no mercado pra quem trabalha com LLM em produto.

In [ ]:
# Importando o PDF
faq_loader = PyPDFLoader("arquivos/faq.pdf")
faq = faq_loader.load()

In [76]:
#dividindo em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

chunk_faq = text_splitter.create_documents(faq)

TypeError: object of type 'PyPDFLoader' has no len()